## Environment

In [205]:
import requests
import json
import pandas as pd

## API

In [206]:
# Twitter API 

def auth():
    token = "AAAAAAAAAAAAAAAAAAAAAP4rkAEAAAAAUKakpY%2B4O%2BIeudDSxtKqdKD2h58%3Dy7HWwhwPXXElUlTiaH7jRbufRtST5CLsLDtwIzjswYKeGsrjtv"
    return token

def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}

    print(query_params)
    print(type(query_params))
    return (search_url, query_params)


#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)

usernames = ['@NorthPowergrid']
#keywords = f"({' OR '.join(usernames)})"
keywords = f"({' OR '.join(usernames)})"

#keyword = "(@NorthPowergrid OR @Northpowergrid OR @northPowergrid OR @northpowergrid)"
keyword = keywords
start_list =    ['2023-03-16T00:00:01.000Z']
end_list =      ['2023-03-16T23:59:59.000Z']

max_results = 500

In [207]:
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

tweets_dict = {}

for i in range(0,len(start_list)):
    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 100 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
#        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']
        

        flag = False
#print(json_response)



-------------------
{'query': '(@NorthPowergrid)', 'start_time': '2023-03-16T00:00:01.000Z', 'end_time': '2023-03-16T23:59:59.000Z', 'max_results': 500, 'expansions': 'author_id,in_reply_to_user_id,geo.place_id', 'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source', 'user.fields': 'id,name,username,created_at,description,public_metrics,verified', 'place.fields': 'full_name,id,country,country_code,geo,name,place_type', 'next_token': {}}
<class 'dict'>
Endpoint Response Code: 200


##Postcode Reg Ex

In [208]:
import re
def code(text):
    pattern = r'\b[A-Z]+(?:\s*[A-Z\d]+)*\b'


    match = re.search(pattern, text)
    if match:
        code=match.group()
    else:
        code=' '
    return code

In [209]:
time,date,text,post_code=[],[],[],[]
for i in range(len(json_response['data'])):
    date.append(json_response['data'][i]['created_at'][:10])
    time.append(json_response['data'][i]['created_at'][11:19])
    text.append(json_response['data'][i]['text'])
    post_code.append(code(json_response['data'][i]['text']))
    

## Data Frame

In [210]:
import pandas as pd

df= pd.DataFrame({"Date": date,"Time":time,"Postcode": post_code,"Tweet_text": text})
display(df)

#df['Postcode'].replace('', np.nan, inplace=True)
#df['Postcode'].replace(' ', np.nan, inplace=True)

df.dropna(subset=['Postcode'], inplace=True)
df.dropna(inplace=True)

#append to csv file
df.to_csv('Dataset.csv', mode='a', header=False)

,Date,Time,Postcode,Tweet_text
0,2023-03-16,19:33:42,,"@Northpowergrid Hi Paul, we keep having power ..."
1,2023-03-16,19:12:41,I,@Northpowergrid Hi Paul. Where can I send comp...
2,2023-03-16,17:09:31,RT,RT @ScarboroughUTC: Yesterday our yr 10 studen...
3,2023-03-16,14:45:31,RT,RT @ScarboroughUTC: Yesterday our yr 10 studen...
4,2023-03-16,13:52:59,RT,RT @MEAHouse: Join @Northpowergrid for their o...
5,2023-03-16,13:18:00,,Join @Northpowergrid for their online event St...
6,2023-03-16,11:04:07,A,Yesterday our yr 10 students took part in mock...
7,2023-03-16,10:20:08,,"@Wightas @Northpowergrid Kev, this is getting ..."
8,2023-03-16,10:02:51,,@Northpowergrid Issue in Armthorpe this mornin...
9,2023-03-16,09:08:23,S7,@Northpowergrid Is the power in sheffield S7 e...


## Clean Data

In [211]:
#cleaning dataframe text column
def text_clean(text):
       
    # removing non-letters
    letters_only = re.sub("[^a-zA-Z]", " ", text)
    
    # converting to lower case, split into individual words
    words = letters_only.lower().split()
        
    # creating a custom list of stopwords then converting it to a set to run faster
    stop_words = ['us', 'http', 'www', 'https', 'com', 'ourselves', 'hers', 'between', 
                  'yourself', 'but', 'there', 'about', 'once', 'during',  
                  'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 
                  'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 
                  'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 
                  'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 
                  'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 
                  'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 
                  'ours', 'had', 'she', 'all', 'at', 'any', 'before', 'them', 
                  'same', 'and', 'been', 'have', 'in', 'will', 'does', 'yourselves', 
                  'then', 'that', 'because', 'what', 'over', 'so', 'can', 'did', 
                  'not', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 
                  'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 
                  't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 
                  'further', 'was', 'here', 'than', 'twitter', 'instagram', 'node', 
                  'pic', 'bit', 'ly', 'utm', 'source', 'ts','hi','gh','rt','yr',]
    stops = set(stop_words)
    
    # removing stop words
    meaningful_words = [w for w in words if not w in stops]
    
    # joining the words back into one string separated by space, 
    # and return the result.
    return(" ".join(meaningful_words))

In [212]:
#initializing an empty list to hold the clean posts
clean_text = []

In [213]:
#setting up a for loop to iterate through the column
j = 0
for text_update in df['Tweet_text']:
    clean_text.append(text_clean(text_update)) 
    j += 1

In [214]:
#pulling the list values into a column
df['clean text'] = clean_text

In [215]:
#verifying that the text was cleaned
df.head()

,Date,Time,Postcode,Tweet_text,clean text
0,2023-03-16,19:33:42,,"@Northpowergrid Hi Paul, we keep having power ...",northpowergrid paul keep power cuts how get kn...
1,2023-03-16,19:12:41,I,@Northpowergrid Hi Paul. Where can I send comp...,northpowergrid paul send complements staff sor...
2,2023-03-16,17:09:31,RT,RT @ScarboroughUTC: Yesterday our yr 10 studen...,scarboroughutc yesterday students took part mo...
3,2023-03-16,14:45:31,RT,RT @ScarboroughUTC: Yesterday our yr 10 studen...,scarboroughutc yesterday students took part mo...
4,2023-03-16,13:52:59,RT,RT @MEAHouse: Join @Northpowergrid for their o...,meahouse join northpowergrid online event stro...


In [216]:
df.shape

(10, 5)

In [217]:
df_dropped = df.dropna()

In [218]:
print(df_dropped)

         Date      Time Postcode  \
0  2023-03-16  19:33:42            
1  2023-03-16  19:12:41        I   
2  2023-03-16  17:09:31       RT   
3  2023-03-16  14:45:31       RT   
4  2023-03-16  13:52:59       RT   
5  2023-03-16  13:18:00            
6  2023-03-16  11:04:07        A   
7  2023-03-16  10:20:08            
8  2023-03-16  10:02:51            
9  2023-03-16  09:08:23       S7   

                                          Tweet_text  \
0  @Northpowergrid Hi Paul, we keep having power ...   
1  @Northpowergrid Hi Paul. Where can I send comp...   
2  RT @ScarboroughUTC: Yesterday our yr 10 studen...   
3  RT @ScarboroughUTC: Yesterday our yr 10 studen...   
4  RT @MEAHouse: Join @Northpowergrid for their o...   
5  Join @Northpowergrid for their online event St...   
6  Yesterday our yr 10 students took part in mock...   
7  @Wightas @Northpowergrid Kev, this is getting ...   
8  @Northpowergrid Issue in Armthorpe this mornin...   
9  @Northpowergrid Is the power in sheffiel

In [219]:
df_dropped.shape

(10, 5)

In [220]:
df.to_csv('Dataset_clean.csv', mode='a', header=True)

## Word2Vec - Preprocessing

In [221]:
%pip install gensim==4.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [222]:
# Importing packages
from __future__ import absolute_import, division, print_function
import gensim.models.word2vec as w2v
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import codecs
import glob
import multiprocessing
import os
import pprint
import re
import nltk
from nltk.corpus import stopwords

#importing warnings to turn off future warnings
import warnings
warnings.simplefilter(action='ignore')

In [223]:
#reading in the data
import pandas as pd
all_tweets_df = pd.read_csv('Dataset_clean.csv')

print(f'Shape of DataFrame: {all_tweets_df.shape}')
all_tweets_df.head()

Shape of DataFrame: (54, 6)


,Unnamed: 0,Date,Time,Postcode,Tweet_text,clean text
0,0.0,2023-03-16,19:33:42,,"@Northpowergrid Hi Paul, we keep having power ...",northpowergrid paul keep power cuts how get kn...
1,1.0,2023-03-16,19:12:41,I,@Northpowergrid Hi Paul. Where can I send comp...,northpowergrid paul send complements staff sor...
2,2.0,2023-03-16,17:09:31,RT,RT @ScarboroughUTC: Yesterday our yr 10 studen...,scarboroughutc yesterday yr students took part...
3,3.0,2023-03-16,14:45:31,RT,RT @ScarboroughUTC: Yesterday our yr 10 studen...,scarboroughutc yesterday yr students took part...
4,4.0,2023-03-16,13:52:59,RT,RT @MEAHouse: Join @Northpowergrid for their o...,meahouse join northpowergrid online event stro...


In [224]:
#checking whether any need to drop rows with missing values
testing = all_tweets_df.dropna()

In [225]:
testing.shape

(50, 6)

In [226]:
testing.head()

,Unnamed: 0,Date,Time,Postcode,Tweet_text,clean text
0,0.0,2023-03-16,19:33:42,,"@Northpowergrid Hi Paul, we keep having power ...",northpowergrid paul keep power cuts how get kn...
1,1.0,2023-03-16,19:12:41,I,@Northpowergrid Hi Paul. Where can I send comp...,northpowergrid paul send complements staff sor...
2,2.0,2023-03-16,17:09:31,RT,RT @ScarboroughUTC: Yesterday our yr 10 studen...,scarboroughutc yesterday yr students took part...
3,3.0,2023-03-16,14:45:31,RT,RT @ScarboroughUTC: Yesterday our yr 10 studen...,scarboroughutc yesterday yr students took part...
4,4.0,2023-03-16,13:52:59,RT,RT @MEAHouse: Join @Northpowergrid for their o...,meahouse join northpowergrid online event stro...


In [227]:
# create series just containing the text column
all_tweets = all_tweets_df['Tweet_text']
print(f'Number of tweets: {len(all_tweets)}')

Number of tweets: 54


In [228]:
print(all_tweets)

0     @Northpowergrid Hi Paul, we keep having power ...
1     @Northpowergrid Hi Paul. Where can I send comp...
2     RT @ScarboroughUTC: Yesterday our yr 10 studen...
3     RT @ScarboroughUTC: Yesterday our yr 10 studen...
4     RT @MEAHouse: Join @Northpowergrid for their o...
5     Join @Northpowergrid for their online event St...
6     Yesterday our yr 10 students took part in mock...
7     @Wightas @Northpowergrid Kev, this is getting ...
8     @Northpowergrid Issue in Armthorpe this mornin...
9     @Northpowergrid Is the power in sheffield S7 e...
10                                           Tweet_text
11    @Northpowergrid Hi Paul, we keep having power ...
12    @Northpowergrid Hi Paul. Where can I send comp...
13    RT @ScarboroughUTC: Yesterday our yr 10 studen...
14    RT @ScarboroughUTC: Yesterday our yr 10 studen...
15    RT @MEAHouse: Join @Northpowergrid for their o...
16    Join @Northpowergrid for their online event St...
17    Yesterday our yr 10 students took part in 

In [229]:
pd.options.display.max_rows

999

In [230]:
pd.options.display.max_rows = 999

In [231]:
type(all_tweets)

pandas.core.series.Series

In [232]:
print(all_tweets)

0     @Northpowergrid Hi Paul, we keep having power ...
1     @Northpowergrid Hi Paul. Where can I send comp...
2     RT @ScarboroughUTC: Yesterday our yr 10 studen...
3     RT @ScarboroughUTC: Yesterday our yr 10 studen...
4     RT @MEAHouse: Join @Northpowergrid for their o...
5     Join @Northpowergrid for their online event St...
6     Yesterday our yr 10 students took part in mock...
7     @Wightas @Northpowergrid Kev, this is getting ...
8     @Northpowergrid Issue in Armthorpe this mornin...
9     @Northpowergrid Is the power in sheffield S7 e...
10                                           Tweet_text
11    @Northpowergrid Hi Paul, we keep having power ...
12    @Northpowergrid Hi Paul. Where can I send comp...
13    RT @ScarboroughUTC: Yesterday our yr 10 studen...
14    RT @ScarboroughUTC: Yesterday our yr 10 studen...
15    RT @MEAHouse: Join @Northpowergrid for their o...
16    Join @Northpowergrid for their online event St...
17    Yesterday our yr 10 students took part in 

In [233]:
type(all_tweets)

pandas.core.series.Series

In [234]:
all_tweets1 = all_tweets.to_frame

In [235]:
type(all_tweets)

pandas.core.series.Series

In [236]:
all_tweets.head()

0    @Northpowergrid Hi Paul, we keep having power ...
1    @Northpowergrid Hi Paul. Where can I send comp...
2    RT @ScarboroughUTC: Yesterday our yr 10 studen...
3    RT @ScarboroughUTC: Yesterday our yr 10 studen...
4    RT @MEAHouse: Join @Northpowergrid for their o...
Name: Tweet_text, dtype: object

In [237]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [238]:
# setting stopwords
stop_words = stopwords.words('english')

In [239]:
# Function to clean up tweets (removing urls, non-alphabetical characters)
def tweets_to_tweetlist(raw):
    clean = re.sub(r"[^a-zA-Z ]", "", raw)
    clean = re.sub(r"http.*?\b","",clean)
    clean = re.sub(r"pictwitter.*?\b","",clean)
    clean = re.sub(r"www.*?\b","",clean)
    clean = re.sub(r'instagramcom.*?\b','',clean)
    return clean

In [240]:
# Iterate through all_tweets, clean tweets and add to tweet_list
tweet_list = []
for tweet in all_tweets:
    tweet_list.append(tweets_to_tweetlist(tweet))
len(tweet_list)

54

In [241]:
# Iterate through tweet_list convert to lowercase and tokenize each tweet
tokenized_tweet_list = [tweet.lower().split(' ') for tweet in tweet_list]   

In [242]:
# Remove stop words from tweets
# final_tweet_list is the version of our tweet list we will use from here on
final_tweet_list = []
for tweet in tokenized_tweet_list:
    word_list = []
    for word in tweet:
        if (word not in stop_words) & (word != ''):
            word_list.append(word)
    final_tweet_list.append(word_list)

In [243]:
#Seeing how many tweets we'll be working with
len(final_tweet_list)

54

In [244]:
# Count number of words in tweets list (all tweets)
token_count = sum([len(tweet) for tweet in final_tweet_list])
print(f"The tweet list contains {token_count} tokens")

The tweet list contains 744 tokens


In [245]:
# Compare different versions of tweets and see how they changed
def compare_tweets(n):    
    print(all_tweets[n]) # original version of nth tweet
    print()
    print(tweet_list[n]) # cleaned version of nth tweet
    print()
    print(tokenized_tweet_list[n]) # tokenized version of nth tweet
    print()
    print(final_tweet_list[n]) # tokenized version of nth tweet (stopwords removed)

In [246]:
#calling the first tweet 
compare_tweets(0)

@Northpowergrid Hi Paul, we keep having power cuts. How do we get to know if Northern power grid are going to find a permanent solution to our problem.

Northpowergrid Hi Paul we keep having power cuts How do we get to know if Northern power grid are going to find a permanent solution to our problem

['northpowergrid', 'hi', 'paul', 'we', 'keep', 'having', 'power', 'cuts', 'how', 'do', 'we', 'get', 'to', 'know', 'if', 'northern', 'power', 'grid', 'are', 'going', 'to', 'find', 'a', 'permanent', 'solution', 'to', 'our', 'problem']

['northpowergrid', 'hi', 'paul', 'keep', 'power', 'cuts', 'get', 'know', 'northern', 'power', 'grid', 'going', 'find', 'permanent', 'solution', 'problem']


## Word2Vec Model

In [247]:
# Instantiate Word2Vec model
tweets2vec = w2v.Word2Vec(
    sg = 1, # skip-gram train algo
    seed = 42, # Random Number Generator to make results repeatable
    workers = multiprocessing.cpu_count(), # number of threads
    vector_size = 300, # Dimensionality of the hidden layer
    min_count = 3, # how many times the word has to appear to be kept in the vocab.
    window = 7, # size of the window to train words
    sample = 1e-5 # downsampling setting for frequent words
)

In [248]:
# Build tweets2vec vocabulary, haven't trained it yet, just loading it into memory
tweets2vec.build_vocab(final_tweet_list)

In [249]:
# how long is our tweets2vec vocabulary?
len(tweets2vec.wv)

95

## Word2Vec Training

In [250]:
# Train tweets2vec model on final_tweet_list
tweets2vec.train(final_tweet_list, total_examples=tweets2vec.corpus_count, epochs=tweets2vec.epochs)

(132, 3720)

In [251]:
# Create directory to store our trained tweets2vec model in
if not os.path.exists("../datasets/training_model"):
    os.makedirs("../datasets/training_model")

In [252]:
# Save our trained tweets2vec model in the trained folder
tweets2vec.wv.save("../datasets/training_model/tweets2vec.w2v")

## t-SNE Model Exploration

In [253]:
# reading in the trained data to explore it
tweets2vec = tweets2vec.wv.load("../datasets/training_model/tweets2vec.w2v")

In [254]:
# Here we are instantiating the t-SNE plot and establishing the parameters. 
tsne = sklearn.manifold.TSNE(n_components = 2, random_state = 42, perplexity= 70,learning_rate = 150,verbose =1,n_iter=5000)

In [255]:
#Coverting to word embeddings
all_word_vectors_matrix = tweets2vec.vectors

In [256]:
# Training it will take some time...
all_word_vectors_matrix_2d = tsne.fit_transform(all_word_vectors_matrix)

[t-SNE] Computing 94 nearest neighbors...
[t-SNE] Indexed 95 samples in 0.001s...
[t-SNE] Computed neighbors for 95 samples in 0.019s...
[t-SNE] Computed conditional probabilities for sample 95 / 95
[t-SNE] Mean sigma: 0.013174
[t-SNE] KL divergence after 250 iterations with early exaggeration: 44.414223
[t-SNE] KL divergence after 850 iterations: 0.227696


In [257]:
#creating a dataframe of the t-SNE coordinates
points = pd.DataFrame(
    [
        (word, coords[0], coords[1])
        for word, coords in [
            (word, all_word_vectors_matrix_2d[tweets2vec.key_to_index.get(word)])
            for word in tweets2vec.index_to_key
        ]
    ],
    columns=["word", "x", "y"]
)

In [258]:
# Edited code from Boom Devahastin Na Ayudhya
# Define vectorization function
def vectorize_corpus(keyword_list):    
    
    # Instantiate counter for number of words in keyword_list that exists
    n_words = 0
    
    # Create template for cumulative corpus vector sum
    corpus_vec_sum = np.zeros((1,300))                 
    
    # Scan through each word in list
    for word in keyword_list:
        if word in tweets2vec:                    
            word_vec = tweets2vec.word_vec(word)        
            n_words +=1                                
            corpus_vec_sum = corpus_vec_sum + word_vec 

    # Compute average vector by taking cumulative vector sum and dividing it by number of words traced
    corpus_avg_vec = corpus_vec_sum/n_words
    
    # Squeeze this N-dimensional nested array object into a 1-D array to streamline future processing
    corpus_avg_vec = np.squeeze(corpus_avg_vec)
    
    return(corpus_avg_vec)

In [259]:
#defining cosine similarity function
def cos_sim(vector_1, vector_2):
    dp = np.dot(vector_1, vector_2)
    magnitude_v1 = np.sqrt(np.dot(vector_1,vector_1))
    magnitude_v2 = np.sqrt(np.dot(vector_2,vector_2))
    return(dp/(magnitude_v1*magnitude_v2))

In [260]:
#creating our pos/neg keyword lists 

# List of words for legitimate outages
power_out = ['generators','flooding','grid', 'massive',
           'failure','fail','generator','darkness','dark','powers','thunderstorms',
           'major', 'serious', 'surge', 'storm','storms','transformer','power','windstorm',
           'substation', 'alert', 'electrical', 'lights','thunderstorm','street','area',
           'lightning','electricity','poweroutage','delays','nopower','lightsout','battery','hour','day','gone','happened','happening','estimate',
           'candlelight','candle','candles','expect','expected','when','power off', 'off','no','elderly','without', 'power cut', 'please', 'ASAP', 'update', 'houses','house','weather', ]

# List of words for non-legitimate outages (such as internet, Netflix, Facebook, or other service outages)
#not_out = ['lines','whatsapp','broadband','thank', 'great news', 'great', 'news', 'great']
not_out = ['involved','thank','you','on','quickly','amazing','friendly','helpful','pleased','appreciate','thanks','much','back']

In [261]:
#vectorizing the keyword lists
power_out_vec = vectorize_corpus(power_out)
not_out_vec = vectorize_corpus(not_out)

In [262]:
#scoring words against our keywords
points['power_out_cs'] = [cos_sim(tweets2vec.word_vec(word),power_out_vec) for word in points['word']]
points['not_out_cs'] = [cos_sim(tweets2vec.word_vec(word),not_out_vec) for word in points['word']]
points['power_out_label'] = np.where(points['power_out_cs'] >= points['not_out_cs'],'power_out','not_out')
points.head()

,word,x,y,power_out_cs,not_out_cs,power_out_label
0,northpowergrid,2.150117,7.667740,-0.019801,-0.035629,power_out
1,yr,1.065153,8.627713,-0.025220,-0.014086,not_out
2,experience,2.599048,8.322472,0.040564,0.114104,not_out
3,allowing,1.818988,7.944089,-0.004271,-0.028318,power_out
4,activity,2.013789,7.163553,0.038178,0.016562,power_out


In [263]:
points.shape

(95, 6)

In [264]:
points.head(100)

,word,x,y,power_out_cs,not_out_cs,power_out_label
0,northpowergrid,2.150117,7.667740,-0.019801,-0.035629,power_out
1,yr,1.065153,8.627713,-0.025220,-0.014086,not_out
2,experience,2.599048,8.322472,0.040564,0.114104,not_out
3,allowing,1.818988,7.944089,-0.004271,-0.028318,power_out
4,activity,2.013789,7.163553,0.038178,0.016562,power_out
5,valuable,1.545115,7.385793,0.002250,-0.049627,power_out
6,really,2.510692,7.780156,0.119577,-0.018769,power_out
7,mock,1.312533,8.044122,-0.004679,-0.085406,power_out
8,part,2.051827,8.748276,0.036094,0.057478,not_out
9,took,1.141135,8.049259,-0.153863,0.019345,not_out


In [265]:
#getting the counts for each label
points['power_out_label'].value_counts()

not_out      48
power_out    47
Name: power_out_label, dtype: int64

In [266]:
#sorting to see what words appear up top for legitimate power outages
points[points['power_out_label'] == 'power_out'].sort_values(by = 'power_out_cs',ascending = False).head(10)

,word,x,y,power_out_cs,not_out_cs,power_out_label
30,without,2.612465,7.925630,0.451908,-0.061382,power_out
40,generator,2.796782,8.209415,0.409352,-0.024098,power_out
59,expected,1.361196,8.249259,0.405025,0.022512,power_out
87,please,2.551913,7.297380,0.360285,0.057845,power_out
14,power,1.926825,8.348298,0.360049,-0.059422,power_out
54,grid,2.446796,7.439237,0.347155,-0.035121,power_out
57,get,2.170888,7.903091,0.142914,0.040994,power_out
66,responsewhich,2.329607,7.666768,0.123954,-0.038592,power_out
6,really,2.510692,7.780156,0.119577,-0.018769,power_out
86,someone,2.063412,8.117365,0.118872,0.046630,power_out


In [267]:
#seeing what the top are for not a legitamate power outage
points[points['power_out_label'] == 'not_out'].sort_values(by = 'not_out_cs',ascending = False).head(10)

,word,x,y,power_out_cs,not_out_cs,power_out_label
28,back,2.271617,8.804829,-0.030001,0.735646,not_out
73,thanks,1.807857,8.185191,-0.029674,0.697509,not_out
48,problem,1.590756,7.528035,-0.010557,0.120753,not_out
27,issue,1.918165,8.489880,0.036666,0.120221,not_out
2,experience,2.599048,8.322472,0.040564,0.114104,not_out
89,reply,1.768460,7.206464,0.042236,0.105470,not_out
72,partners,1.454722,7.866113,0.012727,0.093502,not_out
91,sent,1.809137,7.755376,-0.034997,0.090542,not_out
44,sheffield,1.725522,8.408834,-0.024391,0.090122,not_out
11,interviews,1.771878,7.371253,0.042373,0.088352,not_out


In [268]:
#checking out what words are most similar to the word "power"
tweets2vec.most_similar("power", topn=10)

[('mock', 0.15210062265396118),
 ('issue', 0.12569387257099152),
 ('students', 0.11327991634607315),
 ('scarboroughutc', 0.11291317641735077),
 ('tweettext', 0.11010922491550446),
 ('someone', 0.10363435000181198),
 ('honest', 0.09150104224681854),
 ('online', 0.0805681124329567),
 ('interview', 0.07928049564361572),
 ('per', 0.078421950340271)]

In [269]:
#checking out what words are most similar to the word "outage"
tweets2vec.most_similar("outage")

KeyError: ignored

In [ ]:
#checking out what words are most similar to the word "cut"
tweets2vec.most_similar("cut")

In [ ]:
#checking out what words are most similar to the word "northpowergrid"
tweets2vec.most_similar("northpowergrid")

In [ ]:
#taking a look at each tweet with both scores so we can review what word groups were used to create the score
#this is helpful for fine tuning the model
for tweet in final_tweet_list:
    print(tweet)
    tweet_avg_vec = vectorize_corpus(tweet)
    print(f'Power Out CS: {cos_sim(tweet_avg_vec,power_out_vec)}')
    print(f'Not Out CS: {cos_sim(tweet_avg_vec,not_out_vec)}')

In [ ]:
#creating a dataframe of the scores, including noting areas of blackout
all_tweets_df['score_power_out']     = [cos_sim(vectorize_corpus(tweet),power_out_vec) for tweet in final_tweet_list]
all_tweets_df['score_not_out'] = [cos_sim(vectorize_corpus(tweet),not_out_vec) for tweet in final_tweet_list]
all_tweets_df['blackout']  = np.where(all_tweets_df['score_power_out'] >= all_tweets_df['score_not_out'],'power_out','not_out')
all_tweets_df.head()

In [ ]:
all_tweets_df.shape

In [ ]:
#checking out counts for blackout
all_tweets_df['blackout'].value_counts()

In [ ]:
#saving the csv
all_tweets_df.to_csv('../datasets/final_scored_data.csv')

## Visualisation

In [ ]:
#setting context style
sns.set_context("poster")

In [ ]:
#plotting the t-SNE
plt.figure(figsize = (150,100))
colors = {'power_out':'deepskyblue', 'not_out':'darkgrey'}
for i,word in enumerate(points['word']):
    x = points['x'][i]
    y = points['y'][i]
    color = points['power_out_label'].apply(lambda x: colors[x])[i]
    plt.scatter(x, y, color=color, s = 10)
    plt.text(x+0.01, y+0.01, word,color = color, fontsize=30)
plt.show()

# to see the plot please zoom in. 

## Export Visual

In [ ]:
#creating and exporting an html file of the t-SNE plot

import numpy as np

from bokeh.plotting import figure, show, output_file,ColumnDataSource

TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

source = ColumnDataSource(data = dict(
    x = points['x'],
    y = points['y'],
    label = points['power_out_label'],
    power_out_cs = points['power_out_cs'],
    not_out_cs = points['not_out_cs'],
    radii = ((points['power_out_cs']*points['power_out_label'].apply(lambda x: 1 if (x == 'power_out') else 0))+(points['not_out_cs']*points['power_out_label'].apply(lambda x: 0 if (x == 'power_out') else 1)))*0.25,
    color = points['power_out_label'].apply(lambda x: colors[x]),
    words = points['word']
))
TOOLTIPS =[
    ('word', '@words'),
    ('class','@label'),
    ('power out cos sim','@power_out_cs'),
    ('not out cos sim','@not_out_cs')
]
p = figure(title = 't-SNE Plot of Words in Tweets',
           tooltips=TOOLTIPS)

p.scatter('x', 'y', radius='radii', source = source,
          fill_color='color', line_color = None, fill_alpha=0.3)

output_file("tsne_scatter.html", title="tsne_scatter.py")

show(p)  # open a browser

## Power Cut Map

In [270]:
#importing the packages
from bokeh.plotting import figure, show, output_file, ColumnDataSource,output_notebook
from bokeh.tile_providers import get_provider, Vendors
from bokeh.layouts import column, row, widgetbox, layout
from bokeh.models import Slider, Toggle,DateSlider,DateRangeSlider,sliders,callbacks
from bokeh.models.callbacks import CustomJS
from bokeh.models import SaveTool
from bokeh.io import curdoc,export_svgs
import pandas as pd
from geopy.geocoders import Nominatim
import numpy as np
import matplotlib.pyplot as plt
from datetime import date

In [271]:
#reading in the file
tweet_df = pd.read_csv('../datasets/final_scored_data.csv')
tweet_df.head()


,Unnamed: 0.1,Unnamed: 0,Date,Time,Postcode,Tweet_text,clean text
0,0,0.0,2023-03-16,19:33:42,,"@Northpowergrid Hi Paul, we keep having power ...",northpowergrid paul keep power cuts how get kn...
1,1,1.0,2023-03-16,19:12:41,I,@Northpowergrid Hi Paul. Where can I send comp...,northpowergrid paul send complements staff sor...
2,2,2.0,2023-03-16,17:09:31,RT,RT @ScarboroughUTC: Yesterday our yr 10 studen...,scarboroughutc yesterday yr students took part...
3,3,3.0,2023-03-16,14:45:31,RT,RT @ScarboroughUTC: Yesterday our yr 10 studen...,scarboroughutc yesterday yr students took part...
4,4,4.0,2023-03-16,13:52:59,RT,RT @MEAHouse: Join @Northpowergrid for their o...,meahouse join northpowergrid online event stro...


In [272]:
tweet_df.shape

(32, 7)

In [273]:
testing = tweet_df.drop_duplicates()

In [274]:
testing.shape

(32, 7)

In [275]:
testing.head(500)

,Unnamed: 0.1,Unnamed: 0,Date,Time,Postcode,Tweet_text,clean text
0,0,0.0,2023-03-16,19:33:42,,"@Northpowergrid Hi Paul, we keep having power ...",northpowergrid paul keep power cuts how get kn...
1,1,1.0,2023-03-16,19:12:41,I,@Northpowergrid Hi Paul. Where can I send comp...,northpowergrid paul send complements staff sor...
2,2,2.0,2023-03-16,17:09:31,RT,RT @ScarboroughUTC: Yesterday our yr 10 studen...,scarboroughutc yesterday yr students took part...
3,3,3.0,2023-03-16,14:45:31,RT,RT @ScarboroughUTC: Yesterday our yr 10 studen...,scarboroughutc yesterday yr students took part...
4,4,4.0,2023-03-16,13:52:59,RT,RT @MEAHouse: Join @Northpowergrid for their o...,meahouse join northpowergrid online event stro...
5,5,5.0,2023-03-16,13:18:00,,Join @Northpowergrid for their online event St...,join northpowergrid online event stronger toge...
6,6,6.0,2023-03-16,11:04:07,A,Yesterday our yr 10 students took part in mock...,yesterday yr students took part mock interview...
7,7,7.0,2023-03-16,10:20:08,,"@Wightas @Northpowergrid Kev, this is getting ...",wightas northpowergrid kev getting stupid now ...
8,8,8.0,2023-03-16,10:02:51,,@Northpowergrid Issue in Armthorpe this mornin...,northpowergrid issue armthorpe morning
9,9,9.0,2023-03-16,09:08:23,S7,@Northpowergrid Is the power in sheffield S7 e...,northpowergrid power sheffield expected back o...


In [276]:
tweet

['northpowergrid',
 'is',
 'the',
 'power',
 'in',
 'sheffield',
 's',
 'expected',
 'to',
 'be',
 'back',
 'on',
 'by',
 'pm',
 'today',
 'as',
 'per',
 'website']

In [277]:
#tweet_df.columns= ['Unnamed: 0', 'Date', 'Time', 'postcode', 'text', 'score_power_out','score_not_out', 'blackout']
tweet_df.drop(['Unnamed: 0.1'], axis=1, inplace=True)
tweet_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [278]:
tweet_df.head()

,Date,Time,Postcode,Tweet_text,clean text
0,2023-03-16,19:33:42,,"@Northpowergrid Hi Paul, we keep having power ...",northpowergrid paul keep power cuts how get kn...
1,2023-03-16,19:12:41,I,@Northpowergrid Hi Paul. Where can I send comp...,northpowergrid paul send complements staff sor...
2,2023-03-16,17:09:31,RT,RT @ScarboroughUTC: Yesterday our yr 10 studen...,scarboroughutc yesterday yr students took part...
3,2023-03-16,14:45:31,RT,RT @ScarboroughUTC: Yesterday our yr 10 studen...,scarboroughutc yesterday yr students took part...
4,2023-03-16,13:52:59,RT,RT @MEAHouse: Join @Northpowergrid for their o...,meahouse join northpowergrid online event stro...


In [279]:
#rename column heading with lower case p for postcode
tweet_df.rename(columns={'Postcode':'postcode'}, inplace = True)


In [280]:
tweet_df.shape

(32, 5)

In [281]:
tweet_df.head()

,Date,Time,postcode,Tweet_text,clean text
0,2023-03-16,19:33:42,,"@Northpowergrid Hi Paul, we keep having power ...",northpowergrid paul keep power cuts how get kn...
1,2023-03-16,19:12:41,I,@Northpowergrid Hi Paul. Where can I send comp...,northpowergrid paul send complements staff sor...
2,2023-03-16,17:09:31,RT,RT @ScarboroughUTC: Yesterday our yr 10 studen...,scarboroughutc yesterday yr students took part...
3,2023-03-16,14:45:31,RT,RT @ScarboroughUTC: Yesterday our yr 10 studen...,scarboroughutc yesterday yr students took part...
4,2023-03-16,13:52:59,RT,RT @MEAHouse: Join @Northpowergrid for their o...,meahouse join northpowergrid online event stro...


In [282]:
tweet_df.head(50)

,Date,Time,postcode,Tweet_text,clean text
0,2023-03-16,19:33:42,,"@Northpowergrid Hi Paul, we keep having power ...",northpowergrid paul keep power cuts how get kn...
1,2023-03-16,19:12:41,I,@Northpowergrid Hi Paul. Where can I send comp...,northpowergrid paul send complements staff sor...
2,2023-03-16,17:09:31,RT,RT @ScarboroughUTC: Yesterday our yr 10 studen...,scarboroughutc yesterday yr students took part...
3,2023-03-16,14:45:31,RT,RT @ScarboroughUTC: Yesterday our yr 10 studen...,scarboroughutc yesterday yr students took part...
4,2023-03-16,13:52:59,RT,RT @MEAHouse: Join @Northpowergrid for their o...,meahouse join northpowergrid online event stro...
5,2023-03-16,13:18:00,,Join @Northpowergrid for their online event St...,join northpowergrid online event stronger toge...
6,2023-03-16,11:04:07,A,Yesterday our yr 10 students took part in mock...,yesterday yr students took part mock interview...
7,2023-03-16,10:20:08,,"@Wightas @Northpowergrid Kev, this is getting ...",wightas northpowergrid kev getting stupid now ...
8,2023-03-16,10:02:51,,@Northpowergrid Issue in Armthorpe this mornin...,northpowergrid issue armthorpe morning
9,2023-03-16,09:08:23,S7,@Northpowergrid Is the power in sheffield S7 e...,northpowergrid power sheffield expected back o...


In [283]:
postcodes_list = pd.read_csv('ukpostcodes.csv')
postcodes_list

,id,postcode,latitude,longitude
0,1068625,HG5 9EU,54.017112,-1.466742
1,1068626,HG5 9EW,54.020243,-1.463436
2,1068627,HG5 9HA,54.019417,-1.490354
3,1068628,HG5 9HE,54.021694,-1.499744
4,1068629,HG5 9HF,54.022554,-1.501382
...,...,...,...,...
175912,1047616,HU1 3BP,53.747182,-0.336303
175913,1047618,HU1 3BS,53.747691,-0.331422
175914,1047621,HU1 3DF,53.746766,-0.330306
175915,1047622,HU1 3DG,53.746393,-0.331867


In [284]:
postcodes_list['postcode']= postcodes_list['postcode'].apply(lambda x: x.split(" ")[0])

In [285]:
postcodes_list.head()

,id,postcode,latitude,longitude
0,1068625,HG5,54.017112,-1.466742
1,1068626,HG5,54.020243,-1.463436
2,1068627,HG5,54.019417,-1.490354
3,1068628,HG5,54.021694,-1.499744
4,1068629,HG5,54.022554,-1.501382


In [286]:
tweet_df.head()

,Date,Time,postcode,Tweet_text,clean text
0,2023-03-16,19:33:42,,"@Northpowergrid Hi Paul, we keep having power ...",northpowergrid paul keep power cuts how get kn...
1,2023-03-16,19:12:41,I,@Northpowergrid Hi Paul. Where can I send comp...,northpowergrid paul send complements staff sor...
2,2023-03-16,17:09:31,RT,RT @ScarboroughUTC: Yesterday our yr 10 studen...,scarboroughutc yesterday yr students took part...
3,2023-03-16,14:45:31,RT,RT @ScarboroughUTC: Yesterday our yr 10 studen...,scarboroughutc yesterday yr students took part...
4,2023-03-16,13:52:59,RT,RT @MEAHouse: Join @Northpowergrid for their o...,meahouse join northpowergrid online event stro...


In [287]:
#merging with the postcode file

tweet_df = pd.merge(tweet_df, postcodes_list[['postcode','latitude','longitude']], on='postcode', how='inner')
tweet_df

,Date,Time,postcode,Tweet_text,clean text,latitude,longitude
0,2023-03-16,09:08:23,S7,@Northpowergrid Is the power in sheffield S7 e...,northpowergrid power sheffield expected back o...,53.357160,-1.487407
1,2023-03-16,09:08:23,S7,@Northpowergrid Is the power in sheffield S7 e...,northpowergrid power sheffield expected back o...,53.357156,-1.486461
2,2023-03-16,09:08:23,S7,@Northpowergrid Is the power in sheffield S7 e...,northpowergrid power sheffield expected back o...,53.356800,-1.487261
3,2023-03-16,09:08:23,S7,@Northpowergrid Is the power in sheffield S7 e...,northpowergrid power sheffield expected back o...,53.358342,-1.484163
4,2023-03-16,09:08:23,S7,@Northpowergrid Is the power in sheffield S7 e...,northpowergrid power sheffield expected back o...,53.357367,-1.485376
5,2023-03-16,09:08:23,S7,@Northpowergrid Is the power in sheffield S7 e...,northpowergrid power sheffield expected back o...,53.355459,-1.486797
6,2023-03-16,09:08:23,S7,@Northpowergrid Is the power in sheffield S7 e...,northpowergrid power sheffield expected back o...,53.355631,-1.487095
7,2023-03-16,09:08:23,S7,@Northpowergrid Is the power in sheffield S7 e...,northpowergrid power sheffield expected back o...,53.358727,-1.488020
8,2023-03-16,09:08:23,S7,@Northpowergrid Is the power in sheffield S7 e...,northpowergrid power sheffield expected back o...,53.357097,-1.489436
9,2023-03-16,09:08:23,S7,@Northpowergrid Is the power in sheffield S7 e...,northpowergrid power sheffield expected back o...,53.357502,-1.489582


In [288]:
tweet_df.shape

(900, 7)

In [289]:
tweet_df.drop_duplicates(subset=['postcode'], keep='first', inplace=True, ignore_index=True)


In [290]:
tweet_df.shape

(1, 7)

In [291]:
#confirming that lat/longs imported correctly
tweet_df.head(50)

,Date,Time,postcode,Tweet_text,clean text,latitude,longitude
0,2023-03-16,09:08:23,S7,@Northpowergrid Is the power in sheffield S7 e...,northpowergrid power sheffield expected back o...,53.35716,-1.487407


In [292]:
#looking at counts for blackouts
tweet_df['blackout'].value_counts()

KeyError: ignored

In [293]:
tweet_df.shape

(1, 7)

In [295]:
#function to set x and y for plotting on web mercator
def wgs84_to_web_mercator(df, lon="longitude", lat="latitude"):
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df
tweet_df = wgs84_to_web_mercator(tweet_df)

In [296]:
#sorting values by date
tweet_df.sort_values(by = ['Date'],inplace = True)

In [297]:
#creating columns for day, month, and year from the date column
tweet_df['year'] = tweet_df['Date'].apply(lambda x: int(x.split('-')[0]))
tweet_df['month'] = tweet_df['Date'].apply(lambda x: int(x.split('-')[1]))
tweet_df['day'] = tweet_df['Date'].apply(lambda x: int(x.split('-')[2]))

In [299]:
# Create a dummy column for blackout called power_out
tweet_df['power_out'] = tweet_df['blackout'].apply(lambda x: 1 if x == 'power_out' else 0)

KeyError: ignored

In [300]:
tweet_df.head()

,Date,Time,postcode,Tweet_text,clean text,latitude,longitude,x,y,year,month,day
0,2023-03-16,09:08:23,S7,@Northpowergrid Is the power in sheffield S7 e...,northpowergrid power sheffield expected back o...,53.35716,-1.487407,-165577.422793,7.049338e+06,2023,3,16


In [301]:
tweet_df.shape

(1, 12)

In [302]:
# Group tweets by location, year, and month and sum the total number of tweets classified as
# legitimate power outages for each month

tweet_df_group = tweet_df.groupby(by = ['postcode','year','month'])[['power_out']].sum().reset_index()

KeyError: ignored

In [303]:
lat_dict= dict(zip(tweet_df['postcode'], tweet_df['latitude']))
long_dict= dict(zip(tweet_df['postcode'], tweet_df['longitude']))

In [304]:
# add lat long values to grouped dataframe
tweet_df_group['latitude'] =[lat_dict[i] for i in tweet_df_group['postcode']]

tweet_df_group['longitude'] = [long_dict[i] for i in tweet_df_group['postcode']]


NameError: ignored

In [305]:
# Reproject lat long coordinates form WGS 84 to Web Mercator for plotting points on map
# https://wiki.openstreetmap.org/wiki/Mercator#Python
wgs84_to_web_mercator(tweet_df_group)

NameError: ignored

In [306]:
tweet_df_group.head(5)

NameError: ignored

In [307]:
tweet_df_group.shape

NameError: ignored

## GeoJSON

In [ ]:
pip install pandas-geojson


In [ ]:
import pandas as pd
data = tweet_df_group
print(data.head())

In [ ]:
from pandas_geojson import to_geojson
geo_json = to_geojson(df=data, lat='latitude', lon='longitude',
                 properties=['postcode','year','month',])
print(geo_json)

In [ ]:
from pandas_geojson import write_geojson
write_geojson(geo_json, filename='twitter_map.geojson', indent=4)

## GeoJson Rendering Map

In [ ]:
%pip install geopandas


import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#read in once
df_places = gpd.read_file('twitter_map.geojson')

In [ ]:
df_places['geometry'][0]


In [ ]:
df_places['geometry']

In [ ]:
df_places.plot()